In [1]:
import numpy as np
import math
import sys
from collections import defaultdict
from math import expm1
import time

In [2]:
def computeInitialProb(trainDataFile,numOfStates):
    trainFile=open(trainDataFile,"r")
    metaDataLine = trainFile.readline()
    headerLine = metaDataLine.split(" ")
    numSequences = int(headerLine[0])
    distinctObservations= int(headerLine[1])#Total Number of Distinct Observations
    numOfStates=min(numOfStates,distinctObservations)
    empiricalCount=np.zeros(shape=numOfStates)
    empiricalFreq=defaultdict(int)
    for n in range(numSequences):
        line = trainFile.readline()#Reading Sequences 1 by 1
        l = line.split(" ")
        startState=int(l[1])
        empiricalFreq[startState] = empiricalFreq[startState]+1
    totalObservations=0
    for i in np.arange(numOfStates):
        empiricalCount[i]=empiricalFreq[i]
        totalObservations=totalObservations+empiricalCount[i]
    initialProb=[count/totalObservations for count in empiricalCount]
    return (numOfStates,distinctObservations,initialProb)

In [3]:
def createRandomMatrixA(numOfStates):
    matrixA=np.ndarray(shape=(numOfStates,numOfStates),dtype=float)
    prob=1.0/(numOfStates*numOfStates)
    matrixA.fill(prob)
    return matrixA
def createRandomMatrixB(numOfStates,distinctObservations):
    matrixB=np.ndarray(shape=(numOfStates,distinctObservations),dtype=float)
    prob=1.0/(numOfStates*distinctObservations)
    matrixB.fill(prob)
    return matrixB

In [4]:
def computeAlpha(observations,a,b,pi,alphaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    alphaDpScaleTime0=0
    bTranspose=b.transpose()
    alphaDP[0]=pi*bTranspose[observations[0]]
    alphaDpScaleTime0=np.sum(alphaDP[0])
    alphaDP[0]/=alphaDpScaleTime0
    aTranspose=a.transpose()
    for t in np.arange(1,timePts):
        alphaDpScaleTimeT=0
        for i in np.arange(statesC):
            alphaDP[t][i]=np.sum(alphaDP[t-1]*aTranspose[i])
            alphaDP[t]*=b[i][observations[t]]
        alphaDpScaleTimeT=np.sum(alphaDP[t])
        alphaDP[t]/=alphaDpScaleTimeT
def observationsLikelihood(alphaDP):
    timePts=alphaDP.shape[0]
    ans=0.0
    ans=np.sum(alphaDP[timePts-1])
    return ans

In [5]:
def computeBeta(observations,a,b,pi,betaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    betaDP[timePts-1].fill(1)
    bTranspose=b.transpose()
    for t in np.arange(timePts-2,-1,-1):
        betaDpScaleTimeT=0
        for i in np.arange(statesC):
            betaDP[t][i]=np.sum(a[i]*bTranspose[observations[t+1]]*betaDP[t+1])
        betaDpScaleTimeT=np.sum(betaDP[t])
        betaDP[t]/=betaDpScaleTimeT
    return betaDP

In [6]:
def computeDiGammaDP(alphaDP,betaDP,a,b,observations):
    observationsC=alphaDP.shape[0]
    statesC=alphaDP.shape[1]
    diGammaDP=np.zeros(shape=(statesC,statesC),dtype=float)
    diGammaDenom=observationsLikelihood(alphaDP)
    bTranspose=b.transpose()
    for i in np.arange(statesC):
        for t in np.arange(observationsC-1):
            diGammaDP[i]+=alphaDP[t][i]*a[i]*bTranspose[observations[t+1]]*betaDP[t+1]
    diGammaDP/=diGammaDenom
    return diGammaDP
def computeTransitionProbabilityA(alphaDP,betaDP,a,b,observations):
    statesC=alphaDP.shape[1]
    newlyComputedTransitionProbA=np.zeros(shape=(statesC,statesC),dtype=float)
    diGammaDP=computeDiGammaDP(alphaDP,betaDP,a,b,observations)
    diGammaDPSumGrpByJ=np.apply_along_axis(np.sum,1,diGammaDP)
    for i in np.arange(statesC):    
        if (diGammaDPSumGrpByJ[i]==0):
            newlyComputedTransitionProbA[i]=0.0
        else:
            newlyComputedTransitionProbA[i]=diGammaDP[i]/diGammaDPSumGrpByJ[i]
    return newlyComputedTransitionProbA   

In [14]:
def computeGammaNum(t,j,alphaDP,betaDP):
    return alphaDP[t][j]*betaDP[t][j]
def computeGammaDP(alphaDP,betaDP):
    gammaDenom=observationsLikelihood(alphaDP)
    gammaDP=alphaDP*betaDP
    gammaDP=gammaDP.transpose()
    gammaDP/=gammaDenom
    return gammaDP
def computeTransitionProbabilityB(alphaDP,betaDP,a,b,observations,observationDict):
    statesC=a.shape[0]
    observationsC=b.shape[1]
    newlyComputedObsrProbB=np.zeros(shape=(statesC,observationsC),dtype=float)
    gammaDP=computeGammaDP(alphaDP,betaDP) 
    for i in np.arange(statesC):
        obsrProbDenom =np.sum(gammaDP[i])
        for vk in observationDict:
            newlyComputedObsrProbB[i][vk]=np.sum(gammaDP[i][np.where(observations==vk)])/obsrProbDenom
    return newlyComputedObsrProbB

In [15]:
#Change Convergence Criteria to be more reasonable/Useful
def isConverged(count,convergenceIters):
    if count>=convergenceIters:
        return True
    return False
def Forward_Backward_EM_Algo(observations,A,B,pi,convergenceIters,observationDict):
    count=0
    updatedA=A
    updatedB=B
    while isConverged(count,convergenceIters)==False:
        #Expectation(E)-Step
        alphaDP=np.zeros(shape=(observations.shape[0],updatedA.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        betaDP=np.zeros(shape=(observations.shape[0],updatedA.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        computeAlpha(observations,updatedA,updatedB,pi,alphaDP)
        computeBeta(observations,updatedA,updatedB,pi,betaDP)
        #Maximization(M)-Step
        newA=computeTransitionProbabilityA(alphaDP,betaDP,updatedA,updatedB,observations)
        newB=computeTransitionProbabilityB(alphaDP,betaDP,updatedA,updatedB,observations,observationDict)
        updatedA=newA
        updatedB=newB
        count=count+1
    return (updatedA,updatedB)

In [16]:
def trainHMM(trainDataFile,A,B,pi,convergenceIters,maxSequences=-1):
    trainFile=open(trainDataFile,"r")
    metaDataLine = trainFile.readline()
    headerLine = metaDataLine.split(" ")
    numSequences = int(headerLine[0])
    distinctObservations= int(headerLine[1])#Total Number of Distinct Observations
    observationDict=np.arange(distinctObservations)
    updatedA=np.NaN
    updatedB=np.NaN
    isAUpdated=False
    #for n in range(numSequences):
    if(maxSequences==-1):
        usedSeqs=numSequences
    else:
        usedSeqs=min(maxSequences,numSequences)
    actuallyUsedSeqs=0
    for n in range(usedSeqs):
        line = trainFile.readline()#Reading Sequences 1 by 1
        l = line.split(" ")
        #print("For Sequence ",n," =====================================>")
        if(int(l[0])<=1):
            #print("Skipping ",l)
            continue
        actuallyUsedSeqs+=1
        observations=np.array([int(i) for i in l[1:len(l)]])
        learnedParams=Forward_Backward_EM_Algo(observations,A,B,pi,convergenceIters,observationDict)
        if isAUpdated==False:
            isAUpdated=True
            updatedA=learnedParams[0]
            updatedB=learnedParams[1]
        else:
            updatedA+=learnedParams[0]
            updatedB+=learnedParams[1]
    updatedA=updatedA/actuallyUsedSeqs
    updatedB=updatedB/actuallyUsedSeqs
    return (updatedA,updatedB)

In [17]:
def trainModel(fileLoc,maxNoOfStates,convergenceIters,maxSequences=-1):
    start = time.time()
    initialProbs=computeInitialProb(fileLoc,maxNoOfStates)
    end = time.time()
    print("Computed Initial Prob. in ", end - start ,"seconds")
    pi=initialProbs[2]
    numOfStates=initialProbs[0]
    distinctObservations=initialProbs[1]
    #print(initialProbs)
    A=createRandomMatrixA(numOfStates)
    B=createRandomMatrixB(numOfStates,distinctObservations)
    #print(A)
    #print(B)
    trainedParams=trainHMM(fileLoc,A,B,pi,convergenceIters,maxSequences)
    end=time.time()
    print("For ",maxSequences," Sequences : Total Training Time ",end-start," seconds")
    return trainedParams

In [18]:
%timeit trainModel('Data/1.spice.train.txt',20,7,15)
#(A,B)

Computed Initial Prob. in  0.1694507598876953 seconds
For  15  Sequences : Total Training Time  6.315651178359985  seconds
Computed Initial Prob. in  0.16464495658874512 seconds
For  15  Sequences : Total Training Time  6.409115552902222  seconds
Computed Initial Prob. in  0.18465590476989746 seconds
For  15  Sequences : Total Training Time  7.357304334640503  seconds
Computed Initial Prob. in  0.2495558261871338 seconds
For  15  Sequences : Total Training Time  7.436956167221069  seconds
1 loops, best of 3: 6.41 s per loop


In [19]:
(A,B)=trainModel('Data/1.spice.train.txt',20,7,150)

Computed Initial Prob. in  0.1681218147277832 seconds
For  150  Sequences : Total Training Time  80.80704021453857  seconds


In [ ]:
(A,B)=trainModel('Data/1.spice.train.txt',20,1500)

In [ ]:
#(A,B)=trainModel('Data/1.spice.train.txt',20,1600)
#(A,B)